In [1]:
import streamlit as st
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
import time

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS

from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.environ["GROQ_API_KEY"]

if "vectors" not in st.session_state:

    print("enterd here")

    st.session_state.embedding = OllamaEmbeddings()
    st.session_state.loaders = WebBaseLoader("https://python.langchain.com/v0.2/docs/introduction/")

    st.session_state.docs = st.session_state.loaders.load()

    st.session_state.doc_chunker  = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

    st.session_state.final_documents = st.session_state.doc_chunker.split_documents(st.session_state.docs[:5])

    st.session_state.vectors = FAISS.from_documents(st.session_state.final_documents,st.session_state.embedding)

st.title("chat groq demo")


print("exited here")

llm = ChatGroq(grok_api_key=groq_api_key,model_name="gemma-7b-it")


prompt = ChatPromptTemplate.from_template(
    
""" 
Answer the question based on the context.
Please provide most accurate response.
<context>
{context}
<context>
Question:{input}
                                          
""")

retriever = st.session_state.vectors.as_retriever()
document_chain = create_stuff_documents_chain(llm,prompt)

retrival_chain = create_retrieval_chain(retriever,document_chain)

prompt = st.text_input("Input your prompt here")

if prompt:
    start = time.process_time()
    response = retrival_chain.invoke({"input":prompt})
    print("the process time",time.process_time()-start)

    st.write(response["answer"])

    # with a streamlit expander

    for i,doc in enumerate(response["context"]):

        st.write(doc.page_content)

        st.write("----------------------------------")










USER_AGENT environment variable not set, consider setting it to identify your requests.
2024-06-25 11:14:38.643 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


enterd here
